# EK80 Example

### Setting up project root for module imports in Jupyter Notebook

In [1]:
import os

notebook_path = os.path.abspath("analysis_ek80_long.ipynb")
notebooks_folder_path = os.path.abspath(os.path.join(notebook_path, os.pardir))

project_root = os.path.abspath(os.path.join(notebook_path, os.pardir, os.pardir))

import sys

sys.path.append(project_root)

**Specifies the path to the test data utilized in this notebook:**

In [2]:
oceanstream_ek80_path =  os.path.join(notebooks_folder_path, "test_data", "oceanstream_ek80")

### Imports

In [3]:
import gc
import os
import urllib.request

import echopype.visualize as epviz
import fsspec
import matplotlib.pyplot as plt
import numpy as np

from oceanstream.L0_unprocessed_data import raw_handler, ensure_time_continuity
from oceanstream.L2_calibrated_data import (
    sv_computation,
)

In [4]:
# Load the memory_profiler extension to enable memory usage profiling in the notebook
%load_ext memory_profiler

### Downloads the test files

In [5]:
# Initialize the S3 filesystem using anonymous access
fs = fsspec.filesystem('s3', anon=True)

bucket = "noaa-wcsd-pds"
rawdirpath = "data/raw/Pisces/PC1905/EK80"

# List files in the S3 directory
s3_files = fs.glob(f"{bucket}/{rawdirpath}/*")

# Filter the files based on your criteria (e.g., containing a specific string)
target_files = [s3path for s3path in s3_files if any([f"D2019{dtstr}" in s3path for dtstr in ['0821-T2106']])]

print(f"There are {len(target_files)} target files available")

There are 1 target files available


In [6]:
local_folder = oceanstream_ek80_path

# Create the local folder if it doesn't exist
os.makedirs(local_folder, exist_ok=True)

# Download the filtered files to the local folder
for s3path in target_files:
    file_name = os.path.basename(s3path)
    local_file_path = os.path.join(local_folder, file_name)
    with fs.open(s3path, 'rb') as s3_file, open(local_file_path, 'wb') as local_file:
        local_file.write(s3_file.read())

print(f"Downloaded {len(target_files)} target raw files to {local_folder}")

Downloaded 1 target raw files to /Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80


### Files manipulation

**File finder**

In [7]:
%%time

# Displays all `.raw` files from the "oceanstream_ek80" directory
%memit raw_files_path = raw_handler.file_finder(oceanstream_ek80_path, file_type='raw')
display(raw_files_path)

peak memory: 230.26 MiB, increment: 0.03 MiB


['/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80/D20190821-T210646.raw']

CPU times: user 97.5 ms, sys: 13.6 ms, total: 111 ms
Wall time: 704 ms


**File integrity checker**

In [8]:
%%time

# Verifies if the provided echo sounder files are
# readable by echopype and extracts
# essential metadata such as the campaign ID, date of measurement,
# and sonar model.
raw_files_ck = []
for file_path in raw_files_path:
    %memit raw_files_ck.append(raw_handler.file_integrity_checking(file_path))
    
raw_files_ck

peak memory: 7397.02 MiB, increment: 7166.75 MiB
CPU times: user 50.8 s, sys: 38 s, total: 1min 28s
Wall time: 1min 35s


[{'file_path': '/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80/D20190821-T210646.raw',
  'campaign_id': 'D20190821',
  'date': datetime.datetime(2019, 8, 21, 21, 6, 46),
  'sonar_model': 'EK80',
  'file_integrity': True,
  'use_swap': False}]

**Checked raw files reader**

In [9]:
%%time

# Reads multiple raw echo sounder files and returns a list of Datasets.
%memit aug2019_ds = raw_handler.read_raw_files(raw_files_ck)
aug2019_ds[0]

peak memory: 9591.84 MiB, increment: 9327.36 MiB
CPU times: user 34.9 s, sys: 33.9 s, total: 1min 8s
Wall time: 1min 14s


<EchoData: standardized raw data from Internal Memory>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

### NetCDF files obtained from raw files

**Raw file converter to 'nc'**

In [10]:
%%time

%memit raw_handler.convert_raw_files(raw_files_ck, oceanstream_ek80_path, save_file_type='nc')

peak memory: 8786.23 MiB, increment: 5377.71 MiB
CPU times: user 1min 9s, sys: 41.2 s, total: 1min 50s
Wall time: 2min


**File finder**

In [11]:
%%time

%memit nc_files_path = raw_handler.file_finder(oceanstream_ek80_path, file_type='nc')
nc_files_path

peak memory: 358.64 MiB, increment: 0.26 MiB
CPU times: user 146 ms, sys: 434 ms, total: 580 ms
Wall time: 1.7 s


['/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80/D20190821-T210646.nc']

**File integrity checker**

In [12]:
%%time

nc_files_ck = []
for file_path in nc_files_path:
    %memit nc_files_ck.append(raw_handler.file_integrity_checking(file_path))
    
nc_files_ck

peak memory: 379.60 MiB, increment: 20.93 MiB
CPU times: user 385 ms, sys: 73.3 ms, total: 458 ms
Wall time: 1.04 s


[{'file_path': '/Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80/D20190821-T210646.nc',
  'campaign_id': 'D20190821',
  'date': datetime.datetime(2019, 8, 21, 21, 6, 46),
  'sonar_model': 'EK80',
  'file_integrity': True}]

**Processed files reader**

In [13]:
%%time

%memit ds = raw_handler.read_processed_files(nc_files_path)

peak memory: 384.31 MiB, increment: 19.84 MiB
CPU times: user 400 ms, sys: 68 ms, total: 468 ms
Wall time: 1.01 s


In [14]:
ds = ds[0]
ds

<EchoData: standardized raw data from /Users/simedroniraluca/Documents/pineview/oceanstream/notebooks/test_data/oceanstream_ek80/D20190821-T210646.nc>
Top-level: contains metadata about the SONAR-netCDF4 file format.
├── Environment: contains information relevant to acoustic propagation through water.
├── Platform: contains information about the platform on which the sonar is installed.
│   └── NMEA: contains information specific to the NMEA protocol.
├── Provenance: contains metadata about how the SONAR-netCDF4 version of the data were obtained.
├── Sonar: contains sonar system metadata and sonar beam groups.
│   └── Beam_group1: contains backscatter power (uncalibrated) and other beam or channel-specific data, including split-beam angle data when they exist.
└── Vendor_specific: contains vendor-specific information about the sonar and the data.

### Sv (Volume backscattering strength) computation

In [15]:
%%time

%memit sv = sv_computation.compute_sv(ds, waveform_mode="CW", encode_mode="power")
sv

peak memory: 6761.43 MiB, increment: 6382.47 MiB
CPU times: user 15.1 s, sys: 7.17 s, total: 22.3 s
Wall time: 23.3 s


<xarray.Dataset>
Dimensions:                        (channel: 4, ping_time: 8954,
                                    range_sample: 3890, filenames: 1)
Coordinates:
  * channel                        (channel) <U27 'GPT 0090720346b4-1 ES38B' ...
  * ping_time                      (ping_time) datetime64[ns] 2019-08-21T21:0...
  * range_sample                   (range_sample) int64 0 1 2 ... 3887 3888 3889
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (channel, ping_time, range_sample) float64 ...
    echo_range                     (channel, ping_time, range_sample) float64 ...
    frequency_nominal              (channel) float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               (channel) float64 0.009878 ... 0.05316
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          (channel) float64 ...
    impedance_transducer           (channel) int64 75 75 75 75
    impedance_transceiver          (channel) int64 ...
    receiver_sampling_frequency    (channel) float64 ...
    source_filenames               (filenames) <U111 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:29:39Z
    processing_function:          calibrate.compute_Sv

In [16]:
np.nanmax(sv.echo_range.values)

46.9853424

**'GPT 0090720346b4-1 ES38B' Channel**

In [17]:
selected_channel_38 = 'GPT 0090720346b4-1 ES38B'

In [18]:
selected_channel_data_38 = sv.sel(channel=selected_channel_38)
selected_channel_data_38

<xarray.Dataset>
Dimensions:                        (ping_time: 8954, range_sample: 3890,
                                    filenames: 1)
Coordinates:
    channel                        <U27 'GPT 0090720346b4-1 ES38B'
  * ping_time                      (ping_time) datetime64[ns] 2019-08-21T21:0...
  * range_sample                   (range_sample) int64 0 1 2 ... 3887 3888 3889
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (ping_time, range_sample) float64 nan ... nan
    echo_range                     (ping_time, range_sample) float64 0.0 ... nan
    frequency_nominal              float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               float64 0.009878
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          float64 ...
    impedance_transducer           int64 75
    impedance_transceiver          int64 ...
    receiver_sampling_frequency    float64 ...
    source_filenames               (filenames) <U111 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:29:39Z
    processing_function:          calibrate.compute_Sv

In [19]:
np.nanmax(selected_channel_data_38.echo_range)

46.9732608

In [20]:
selected_channel_data_38.ping_time

<xarray.DataArray 'ping_time' (ping_time: 8954)>
array(['2019-08-21T21:06:46.976054784', '2019-08-21T21:06:47.201069056',
       '2019-08-21T21:06:47.417081856', ..., '2019-08-21T21:41:42.200756224',
       '2019-08-21T21:41:42.450770944', '2019-08-21T21:41:42.699785216'],
      dtype='datetime64[ns]')
Coordinates:
    channel    <U27 'GPT 0090720346b4-1 ES38B'
  * ping_time  (ping_time) datetime64[ns] 2019-08-21T21:06:46.976054784 ... 2...

In [21]:
selected_channel_data_38.ping_time[0]

<xarray.DataArray 'ping_time' ()>
array('2019-08-21T21:06:46.976054784', dtype='datetime64[ns]')
Coordinates:
    channel    <U27 'GPT 0090720346b4-1 ES38B'
    ping_time  datetime64[ns] 2019-08-21T21:06:46.976054784

In [22]:
np.nanmax(selected_channel_data_38.sel(ping_time='2019-08-21T21:06:46.976054784').echo_range.values)

22.9067136

In [23]:
selected_channel_data_38.sel(ping_time='2019-08-21T21:06:46.976054784').Sv.values

array([nan, nan, nan, ..., nan, nan, nan])

In [24]:
np.nanmax(selected_channel_data_38.isel(ping_time= 3890).echo_range)

27.4010688

**'GPT 009072058cb4-1 ES120-7C' Channel**

In [25]:
selected_channel_120 = 'GPT 009072058cb4-1 ES120-7C'

In [26]:
selected_channel_data_120 = sv.sel(channel=selected_channel_120)
selected_channel_data_120

<xarray.Dataset>
Dimensions:                        (ping_time: 8954, range_sample: 3890,
                                    filenames: 1)
Coordinates:
    channel                        <U27 'GPT 009072058cb4-1 ES120-7C'
  * ping_time                      (ping_time) datetime64[ns] 2019-08-21T21:0...
  * range_sample                   (range_sample) int64 0 1 2 ... 3887 3888 3889
  * filenames                      (filenames) int64 0
Data variables: (12/24)
    Sv                             (ping_time, range_sample) float64 nan ... ...
    echo_range                     (ping_time, range_sample) float64 0.0 ... ...
    frequency_nominal              float64 ...
    sound_speed                    float64 1.51e+03
    sound_absorption               float64 0.03779
    temperature                    float64 10.0
    ...                             ...
    beamwidth_athwartship          float64 ...
    impedance_transducer           int64 75
    impedance_transceiver          int64 ...
    receiver_sampling_frequency    float64 ...
    source_filenames               (filenames) <U111 '/Users/simedroniraluca/...
    water_level                    float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.1.dev2337+g364c06a
    processing_time:              2023-11-21T15:29:39Z
    processing_function:          calibrate.compute_Sv

In [27]:
np.nanmax(selected_channel_data_120.echo_range)

46.9853424

In [28]:
np.nanmax(selected_channel_data_120.isel(ping_time= 3890).echo_range)

27.4010688

In [29]:
np.nanmax(selected_channel_data_120.isel(ping_time= 0).echo_range)

22.95504

In [30]:
np.nanmax(selected_channel_data_120.isel(ping_time= 8953).echo_range)

46.9853424